Units never move or attack diagonally.
When multiple choices are equally valid, ties are broken in reading order:
+ top-to-bottom, then left-to-right

Each unit begins its turn by identifying all possible targets (enemy units). If no targets remain, combat ends.

```
Targets:      In range:     Reachable:    Nearest:      Chosen:
#######       #######       #######       #######       #######
#E..G.#       #E.?G?#       #E.@G.#       #E.!G.#       #E.+G.#
#...#.#  -->  #.?.#?#  -->  #.@.#.#  -->  #.!.#.#  -->  #...#.#
#.G.#G#       #?G?#G#       #@G@#G#       #!G.#G#       #.G.#G#
#######       #######       #######       #######       #######
```
In the above scenario, the Elf has three targets (the three Goblins):

+ Each of the Goblins has open, adjacent squares which are in range (marked with a ? on the map).
+ Of those squares, four are reachable (marked @); the other two (on the right) would require moving through a wall or unit to reach.
+ Three of these reachable squares are nearest, requiring the fewest steps (only 2) to reach (marked !).
+ Of those, the square which is first in reading order is chosen (+).

Each unit begins its turn by identifying all possible targets (enemy units). If no targets remain, combat ends.

```
In range:     Nearest:      Chosen:       Distance:     Step:
#######       #######       #######       #######       #######
#.E...#       #.E...#       #.E...#       #4E212#       #..E..#
#...?.#  -->  #...!.#  -->  #...+.#  -->  #32101#  -->  #.....#
#..?G?#       #..!G.#       #...G.#       #432G2#       #...G.#
#######       #######       #######       #######       #######
```

In [24]:
from typing import Tuple, List

import collections
import re 
import numpy as np


def sum_tuple(t1: Tuple[int, int], t2: Tuple[int, int]) -> Tuple[int, int]:
    return tuple(map(sum, zip(t1, t2)))


def parse(f: str) -> np.ndarray:
    with open(f) as f:
        raw = f.readlines()
    raw = [l.replace('\n', '') for l in raw]  # removes end lines
    arr = np.asanyarray([list(l) for l in raw], dtype='<U10')
    return arr


def print_array(arr):
    print('\n'.join(' '.join(f'{x}' for x in line) for line in arr))


def bfs(grid, start, goal):
    """Breadth First Search"""
    wall, clear = "#", "."
    width, height = grid.shape
    queue = collections.deque([[start]])
    seen = set([start])
    while queue:
        path = queue.popleft()
        x, y = path[-1]
        if grid[y, x] == goal:  # change to coordinates
            return path
        for x2, y2 in ((x+1,y), (x-1,y), (x,y+1), (x,y-1)):
            if 0 <= x2 < width and 0 <= y2 < height and grid[x2, y2] != wall and (x2, y2) not in seen:
                queue.append(path + [(x2, y2)])
                seen.add((x2, y2))


class Unit():
    
    hitpoints: int
    attack_power: int
    kind: str
    pos: Tuple[int, int]
    turn: int

    def __init__(self, kind, pos):
        
        self.kind = kind
        self.attack_power = 3
        self.hitpoints = 200
        self.kind_long = self.expand_kind()
        self.pos = pos
        self.turn = 0

    def expand_kind(self):
        if self.kind == 'E':
            long = 'Elf'
        else:
            long = 'Goblin'
        return long
    
    def __repr__(self):
        rep = f'Unit({self.kind_long}, {self.hitpoints}, {self.pos}, {self.turn})'
        return rep
    
    def find_targets(self, units) -> List:
        """Find closest enemy target"""
        if self.kind == 'E':
            target = 'G'
        else:
            target = 'E'
        
        self.targets = [x for x in units if x.kind == target]
        return self.targets
    
    @staticmethod
    def look_around(pos, board) -> List[Tuple[str, Tuple[int, int]]]:
        """Returns the chars around a point"""
        # up, rigth, down, left
        looks = ((1,0), (0,1), (-1,0), (0,-1))
        chars = [(board[sum_tuple(pos, l)],sum_tuple(pos, l)) for l in looks]
        return chars
    
    def bfs(grid, start):
        wall, clear, goal = "#", ".", "*"
        queue = collections.deque([[start]])
        seen = set([start])
        while queue:
            path = queue.popleft()
            x, y = path[-1]
            if grid[y, x] == goal:
                return path
            for x2, y2 in ((x+1,y), (x-1,y), (x,y+1), (x,y-1)):
                if 0 <= x2 < width and 0 <= y2 < height and grid[y2][x2] != wall and (x2, y2) not in seen:
                    queue.append(path + [(x2, y2)])
                    seen.add((x2, y2))
                    
    def is_reachable(self, target) -> bool:
        pass
        
    
    def find_empty_squares(self, board):
        for target in self.targets:
            # if self.is_reachable(target):
            # reachable
            sorroundings = self.look_around(target.pos, board)
            # in range and not wall
            sorroundings = [c for c in sorroundings if c[0] != '#']
    
            print(sorroundings)
        

In [25]:
board = parse('day15_example_walled.txt')

In [26]:
path = bfs(board, (1,1), 'E')

In [27]:
print_array(board)

# # # # # # # # #
# G . . G . . G #
# . . . . . . . #
# . . # # # # # #
# G . # E . . G #
# . . # # . . . #
# . . . . . . . #
# G . . G . . G #
# # # # # # # # #


In [28]:
for p in path:
    board[p] = '*'
print_array(board)

# # # # # # # # #
# * . . G . . G #
# * . . . . . . #
# * . # # # # # #
# * . # * * . G #
# * . # # * . . #
# * * * * * . . #
# G . . G . . G #
# # # # # # # # #


In [29]:
def find_units(board: np.ndarray) -> List[Unit]:
    units = []
    goblins = list(tuple(zip(*np.where(board == 'G'))))
    elfs = list(tuple(zip(*np.where(board == 'E'))))
    
    for pos in goblins:
        goblin = Unit('G', pos)
        units.append(goblin)
    
    for pos in elfs:
        elf = Unit('E', pos)
        units.append(elf)
    
    units = sorted(units, key=lambda x: (x.pos[0], x.pos[1]))
    
    for i, u in enumerate(units):  # put original turn 
        u.turn = i
    return units 

In [30]:
units = find_units(board)

In [31]:
units[0].find_targets(units)

[]

In [23]:
units[0].find_empty_squares(board)